In [ ]:
import sys
sys.path.append("..")

import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as col
import numpy as np
import json
# import geopandas as gpd
# import geoplot as gpt

from shapely.geometry import Point, Polygon
from dotenv import load_dotenv

import requests
import os
import math
import pandas_gbq

load_dotenv()

# --------- OSM ---------

from OSMPythonTools.api import Api  
from OSMPythonTools.nominatim import Nominatim # Tool to search OSM data by name and address
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass # read-only API that serves up custom selected parts of the OSM map data

from sid_modules.helperFunctions import BigQueryHelper, OSMQueryHelper

bqh = BigQueryHelper()
osh = OSMQueryHelper()

In [ ]:
### **Districts of Nikki** (County)

In [ ]:
# Plot the districts 
district_df = bqh.loadDistricts()

district_geodf = gpd.GeoDataFrame(district_df, geometry="polygon")
district_geodf.rename(columns={"polygon": "geometry"}, inplace=True)
fig, ax = plt.subplots(1, 1,figsize=(15,15))

district_geodf.plot(column="name",  ax=ax, legend=False, cmap='Blues', figsize=(15,15))


In [ ]:
### **Villages in Nikki** (County)

In [ ]:
# Get all villages in OSM
new_village_df = osh.getVillagesFromOSM()

# Show number of villages extracted
new_village_df.shape

In [ ]:
# Load existing villages from BigQuery
village_df = bqh.loadVillages()

# Show number of villages currently in BigQuery
village_df.shape

In [ ]:
# Add missing villages in BigQuery df from the new villages in OSM
for index, new_row in new_village_df.iterrows():
  if not osh.isIdalready(village_df, new_row['osm_id']) :
    village_df = village_df.append(new_row)

# Update missing areas id
osh.updateAreaIdOfDataframe(village_df)

# Update missing buildings numbers
# osh.getBuildingsInVillage(village_df)

# Update county origin
osh.updateDistrictOfVillage(district_df, village_df)

In [ ]:
### **Buildings of Nikki** (County)

In [ ]:
# Villages selectect for POC
villages_selected_id = [
    377129182, # MONNON
    737083662, # SANSI GANDÓ
    738644153, # BESEN GOUROU
    738657269, # BARKEDJE BARIKEDJE
    745997907, # BOUDAL
]

In [ ]:
building_headers = ['osm_id','type','district','village','lat', 'lon','perim','area','boundary_lat','boundary_lon','polygon', 'point']
new_buildings_data = []

# Loop throught all selected villages for the POC
for vill_id in villages_selected_id:

    # Extract the data from the current village
    row = village_df.loc[village_df['osm_id'] == vill_id ]

    # Get all the buildings inside the village
    buildings = osh.get_buildings_in_village(row)
    # Add all new buildings to the df data
    new_buildings_data.extend(buildings)

# Create Dataframe from data
new_building_df = pd.DataFrame(new_buildings_data,   columns=building_headers)
new_building_df.shape

In [ ]:
# Load existing buildings from BigQuery
building_df = bqh.loadOSMBuildings()

# Show number of buildings currently in BigQuery
building_df.shape

In [ ]:
# Add missing buildings in BigQuery df from the new buildings in OSM
for index, new_row in new_building_df.iterrows():
  if not osh.isIdalready(building_df,new_row['osm_id']) :
    building_df = building_df.append(new_row)

building_df.shape

In [ ]:
# Save buildings
saveBuildings(building_df)